In the lecture, we saw an example of forward mode auto-differentiation. 

Another way to compute derivatives using autodiff is the so-called backward mode.

In the backward mode, there are two passes - forward and backward.

<br>
<br>


<div>
    <center>
        <img src="figures/reverse_mode_autodiff.png" width="1000"/>
    </center>
</div>

<br>
<br>

Notice how, in just one forward+backward pass, we compute the derivative with respect to `x1` and `x2`.

This is contrary to forward mode, where we need one pass per differentiation wrt `x1` and another pass wrt `x2`.

$\textbf{Exercise}$:

Re-write the differentiation function `fn_prime(x1,x2)` for the funtion `fn(x1, x2)` defined below using reverse-mode differentiation concepts. 

Do this by calculating the adjoints. 

In [117]:
import jax
from jax import numpy as jnp

import numpy as np

# This is the same function as the lecture

def fn(x1, x2):
    
    a = x1/x2      
    b = jnp.exp(x2)     
    c = jnp.sin(a)
    d = a - b
    e = c + d
    g = d * e
    
    return g

In [ ]:
# Write down the function

def fn_prime(x1, x2):
    
    

Once you write this down, compare the outputs with the `jax.grad` method.

In [ ]:
print("Your output")
print(fn_prime(1.0,1.0))

print("JAX output")
print(jax.grad(fn, argnums=(0,1))(1.0,1.0))

<br>

A common step in doing a statistical analysis is calculating the Hessian matrix for a negative log-likelihood function.

The Hessian matrix of a function, as you might be aware, is a matrix of second derivatives of the function wrt it's parameters:

<br>
<br>


<div>
    <center>
        <img src="figures/hessian.png" width="500"/>
    </center>
</div>

<br>
<br>

The dimensions of a Hessian matrix is thus equal to the number of parameters of the function. 

The more parameters we have, the larger it is. In ML models, these parameters can range from thousands to billions!

This is why in most ML optimizations, a Hessian matrix is not computed by default.

<br>

But we are not intereseted in ML applications in these set of lectures. 

Hessian matrix is also computed for parameter fitting in the profile likelihood fit step of a typical statistical analysis. 

This Hessian matrix is typically $O(100)\times O(100)$

We will explore this in the next lecture, but for now let us try and calculate Hessian matrices for arbitrary functions!

<br>

Let's say we have the following two (toy) function:


In [137]:
def fn(tuple_arr):
    
    x = 1.0
    
    for param in tuple_arr:
        
        x *= (param**2-param**3-param)
    
    return x

# def fn(tuple):
    
#     z = 1.0
    
#     y = (tuple[1]**2-tuple[1]**3-tuple[1])*23+5.0
    
#     z = z*y
        
#     x = (tuple[0]**2-tuple[0]**3-tuple[0])*23+5.0
    
#     z = z*x
    
#     return z

Using the `jax.hessian` method, try calculating the Hessian matrix of this function at the point $(x,y) = (1,0)$

Documentation: https://jax.readthedocs.io/en/latest/_autosummary/jax.hessian.html

In [138]:
# Calculate the Hessian using the jax.hessian method

from jax import hessian

hess_mat = hessian(fn)(np.array([5.0,5.0]))

print(hess_mat)

[[2940. 4356.]
 [4356. 2940.]]


Now compute the hessian for a 100-dimensional array of randomly assigned numberse between 0 and 1

In [140]:
# Calculate the Hessian using the jax.hessian method

hess_mat = hessian(fn)(np.random.uniform(1.0,5.0,size=100))

hess_mat = hessian(fn)(np.ones(100))

print(hess_mat)

[[4. 4. 4. ... 4. 4. 4.]
 [4. 4. 4. ... 4. 4. 4.]
 [4. 4. 4. ... 4. 4. 4.]
 ...
 [4. 4. 4. ... 4. 4. 4.]
 [4. 4. 4. ... 4. 4. 4.]
 [4. 4. 4. ... 4. 4. 4.]]


<br>

Well that was easy, right? JAX is a powerful and easy to use tool. But we have just scratched the surface of it's capabilities.

Now let's explore a bit how the Hessian is calculated under the hood (https://jax.readthedocs.io/en/latest/_modules/jax/_src/api.html#hessian):

<br>

```
def hessian(fun: Callable, argnums: Union[int, Sequence[int]] = 0,
            has_aux: bool = False, holomorphic: bool = False) -> Callable:
            
            
    return jacfwd(jacrev(fun, argnums, has_aux=has_aux, holomorphic=holomorphic),
                argnums, has_aux=has_aux, holomorphic=holomorphic)
```

It seems that it is calculating the full Hessian matrix by first performing a reverse mode differentiation, and then performing a forward mode differentiation ont he output.

For the scalar valued function we use, this looks like

$$\nabla f : \mathbb{R}^2 \rightarrow \mathbb{R}^2  \\ \text{(Calculated using Reverse Mode)}$$ 

<br>

$$\nabla(\nabla f) : \mathbb{R}^2 \rightarrow \mathbb{R}^{2 \times 2} \\ \text{(Calculated using Forward Mode)}$$ 

Is this the most efficient way of computation? Why not use jacfwd for both steps? Or reverse mode? Or forward mode followed by reverse mode?

Let's try it out. Let's calculate the computation times

In [106]:
%%timeit -r10 -n10

#Below write down the simple hessian function calculated for f(1.0,1.0)

jax.hessian(fn)(jnp.array(np.ones(200))).block_until_ready()

311 ms ± 2.12 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [105]:
%%timeit -r10 -n10

#Below compute the hessian using a combination of forward mode differentiations only 

jax.jacfwd(jax.jacfwd(fn))(jnp.array(np.ones(200))).block_until_ready()

305 ms ± 2.31 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [104]:
%%timeit -r10 -n10

#Below compute the hessian using a combination of reverse mode differentiations only 

jax.jacrev(jax.jacrev(fn))(jnp.array(np.ones(200))).block_until_ready()

296 ms ± 2.2 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [103]:
%%timeit -r10 -n10

#Below compute the hessian after reversing the order of forward and backward modes in the jax hessian method

jax.jacrev(jax.jacfwd(fn))(jnp.array(np.ones(200))).block_until_ready()

324 ms ± 10.6 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [102]:
%%timeit -r10 -n10

#Below compute the hessian after reversing the order of forward and backward modes in the jax hessian method

jax.jacfwd(jax.jacrev(fn))(jnp.array(np.ones(200))).block_until_ready()

314 ms ± 1.32 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [149]:
%%timeit -r1 -n1

jax.jacrev(fn)(jnp.array(np.ones(1000))).block_until_ready()

46 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [150]:
%%timeit -r1 -n1

jax.jacfwd(fn)(jnp.array(np.ones(1000))).block_until_ready()

26.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
print(jax.jacrev(fn)(jnp.array(np.ones(200))))
print(jax.jacfwd(fn)(jnp.array(np.ones(200))))